In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Major_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [4]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [5]:
# Create a df for more than 20 total_votes
votes_greater_20_df = vine_df.filter(vine_df.total_votes > 20)
votes_greater_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|     

In [7]:
# Filter the new DataFrame and create a new DataFrame to retrieve all the rows where 
# the number of helpful_votes divided by total_votes is equal to or greater than 50%
divided_df = votes_greater_20_df.filter((votes_greater_20_df.helpful_votes / votes_greater_20_df.total_votes) >= 0.5)
divided_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|                Y|
|R1A4JVOP7N5Q1K|          5|           28|         30|   N|     

In [8]:
# Vine = yes DataFrame
vine_yes_df = divided_df.filter(divided_df.vine == "Y")
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|
| RCNB7YGLWKCV5|          5|           23|         30|   Y|                N|
| RX1ZUVGBH3HKC|          5|          814|        848|   Y|                N|
| RRS5AGL10DYU3|          3|           25|         29|   Y|                N|
| R5YFC5G8V2B3I|          5|          104|        117|   Y|                N|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|
| RKXNZX22HQZEE|          3|          182|        192|   Y|                N|
|R2K45IY0NO70ZI|          5|          116|        122|   Y|                N|
| RQXRI4UKDX877|          5|           20|         24|   Y|                N|
|R1L2R8OHO171YG|          5|           66|         68|   Y|     

In [9]:
# Vine = No DataFrame
vine_no_df = divided_df.filter(divided_df.vine == "N")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|                Y|
|R3FAFI6Q0YL37W|          1|           30|         35|   N|                Y|
|R1A4JVOP7N5Q1K|          5|           28|         30|   N|     

In [12]:
# Yes df total reviews
vine_yes_total = vine_yes_df.count()
vine_yes_total

31

In [17]:
# Yes df 5 start reviews
vine_yes_5star = vine_yes_df.filter(vine_yes_df.star_rating == 5)
vine_yes_5star.count()

17

In [19]:
# Yes df percentage of 5 star
vine_yes_percentage = vine_yes_5star.count() / vine_yes_total * 100
vine_yes_percentage

54.83870967741935

In [20]:
# No df total reviews
vine_no_total = vine_no_df.count()
vine_no_total

4655

In [21]:
# No df 5 start reviews
vine_no_5star = vine_no_df.filter(vine_no_df.star_rating == 5)
vine_no_5star.count()

1865

In [22]:
# No df percentage of 5 star
vine_no_percentage = vine_no_5star.count() / vine_no_total * 100
vine_no_percentage

40.06444683136412